
Conectar drive personal

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Importar librerías necesarias e instalar modelo para modelado de tópicos BERTopic

In [ ]:
import pandas as pd
!pip install bertopic

Importar modelo BERTopic e instanciarlo con parámetros de configuración básicos: lenguaje, cálculo de probabilidad de los tópicos obtenidos por documento y seguimiento de las fases del modelo (verbose)

In [ ]:
#IMPORTAR MODELO
from bertopic import BERTopic
topic_model = BERTopic(language="english", calculate_probabilities=False, verbose=True)

Primero se lee el dataset con los tweets limpios y pre-procesados. A continuación, se transforma la columna 'body' con los tweets a una lista de strings para utilizarla como entrada al modelo BERTopic. Finalmente, se obtienen los tópicos más relevantes del tópico

In [ ]:
#leer dataset
data_tesla = pd.read_csv('/content/drive/MyDrive/TFG ADE/Data_Tesla_Limpio.csv')
data=(data_tesla['body'].astype(str)).tolist()
topics, _ = topic_model.fit_transform(data)

Batches:   0%|          | 0/5988 [00:00<?, ?it/s]

2023-11-14 17:43:30,801 - BERTopic - Transformed documents to Embeddings
2023-11-14 17:51:43,932 - BERTopic - Reduced dimensionality
2023-11-14 17:52:03,969 - BERTopic - Clustered reduced embeddings


Se obtiene información general sobre los tópicos obtenidos por el modelo: nombre/identificador, términos representativos y documentos representativos (-1 es el tópico outlier, no se tiene en cuenta en el análisis)

In [ ]:
freq = topic_model.get_topic_info()
freq

,Topic,Count,Name,Representation,Representative_Docs
0,-1,85381,-1_autopilot_musk_car_battery,"[autopilot, musk, car, battery, elonmusk, sale...",[stock value isnt useful whats interesting fla...
1,0,2116,0_solar_roof_solarcity_tile,"[solar, roof, solarcity, tile, city, panel, ba...","[solar panel roof, car battery solar roof, sol..."
2,1,992,1_nan_nano_ching_yeah,"[nan, nano, ching, yeah, , , , , , ]","[nan, nan, nan]"
3,2,980,2_gspc_surge_fall_outlook,"[gspc, surge, fall, outlook, nflx, soar, rise,...","[gspc nflx share rise earnings beat, gspc nflx..."
4,3,884,3_elon_elons_stuttering_tapp,"[elon, elons, stuttering, tapp, lie, wright, g...","[elon, elon, elon]"
...,...,...,...,...,...
2122,2121,10,2121_signal_issued_potnetial_outpretty,"[signal, issued, potnetial, outpretty, abux, n...",[weekly signal issued monday amfor solid move ...
2123,2122,10,2122_burger_sandwich_restocked_pbhto,"[burger, sandwich, restocked, pbhto, bintsla, ...","[go month bynd burger, coming burger, burger]"
2124,2123,10,2123_localnational_fortuitous_extensionbased_b...,"[localnational, fortuitous, extensionbased, bu...",[anyone show sale increased localnational gove...
2125,2124,10,2124_4200_yuasa_jvor_whyteslas,"[4200, yuasa, jvor, whyteslas, bydgf, jvaesc, ...",[go time high believe achieve 4200 share year ...


Ya que se obtienen un gran número de documentos pertenecientes al tópico -1 (outlier), se reducen los mismos y se actualizan las representaciones de los tópicos obtenidos

In [ ]:
new_topics = topic_model.reduce_outliers(data, topics)
topic_model.update_topics(data, topics=new_topics)

Se obtiene la información general sin tantos outliers

In [ ]:
freq = topic_model.get_topic_info()
freq

,Topic,Count,Name,Representation,Representative_Docs
0,-1,192,-1_oniontsla_tslaentry_illegaltsla_caching,"[oniontsla, tslaentry, illegaltsla, caching, t...",[stock value isnt useful whats interesting fla...
1,0,2180,0_solar_roof_solarcity_tile,"[solar, roof, solarcity, tile, city, panel, ba...","[solar panel roof, car battery solar roof, sol..."
2,1,992,1_nan_nano_ching_yeah,"[nan, nano, ching, yeah, , , , , , ]","[nan, nan, nan]"
3,2,1003,2_gspc_surge_fall_outlook,"[gspc, surge, fall, outlook, nflx, soar, rise,...","[gspc nflx share rise earnings beat, gspc nflx..."
4,3,905,3_elons_elon_stuttering_tapp,"[elons, elon, stuttering, tapp, lie, wright, g...","[elon, elon, elon]"
...,...,...,...,...,...
2122,2121,18,2121_issued_signal_yesterdaystock_alreadyspy,"[issued, signal, yesterdaystock, alreadyspy, t...",[weekly signal issued monday amfor solid move ...
2123,2122,31,2122_burger_sandwich_ribt_vegitarian,"[burger, sandwich, ribt, vegitarian, mash, che...","[go month bynd burger, coming burger, burger]"
2124,2123,54,2123_sticker_restriction_extended_household,"[sticker, restriction, extended, household, on...",[anyone show sale increased localnational gove...
2125,2124,18,2124_4200_dealt_obsolete_yuasa,"[4200, dealt, obsolete, yuasa, mitsubishigs, w...",[go time high believe achieve 4200 share year ...


Se extraen los n términos top del tópico más relevante/repetido (0) y la puntuación c-TF-IDF de cada uno

In [ ]:
topic_model.get_topic(0)

[('solar', 0.01760027467952953),
 ('roof', 0.017043836447936053),
 ('solarcity', 0.01638360065103524),
 ('tile', 0.010647335229350924),
 ('city', 0.008134965581446704),
 ('panel', 0.007029349518304793),
 ('bailout', 0.005115056075276034),
 ('installation', 0.004787304330391724),
 ('shingle', 0.004786569358671893),
 ('acquisition', 0.003870052534784978)]

Debido al elevado número de tópicos encontrado (2127) y la limitación computacional, no es posible representar la distancia intertópica inicial

In [ ]:
#DISTANCIA INTERTOPICA INICIAL
topic_model.visualize_topics()

Se guarda el modelo para no perder la clasificación inicial cuando expire la sesión

In [ ]:
topic_model.save('/content/drive/MyDrive/TFG ADE/teslatopicinicial')

Se agrupan los tópicos para obtener las relaciones existentes entre ellos y se visualiza la jerarquía de los 25 más relevantes, con el objetivo de reducir el número de tópicos

In [ ]:
#JERARQUIA INICIAL QUE AYUDA A REDUCIR EL NUMERO DE TOPICOS
topic_model.visualize_hierarchy(top_n_topics=25)

Se vuelve a cargar el modelo

In [ ]:
modelo = BERTopic.load('/content/drive/MyDrive/TFG ADE/teslatopicinicial')

Se van reduciendo el número de tópicos y visualizando la distancia intertópica hasta obtener el número óptimo (20) en el que la distancia es lo suficientemente grande como para que estos sean diferenciables

Se van reduciendo el número de tópicos y visualizando la distancia intertópica hasta obtener el número óptimo (22) en el que la distancia es lo suficientemente grande como para que estos sean diferenciables

In [ ]:
#PRUEBAS TOPIC REDUCTION Y DISTANCIA INTERTOPICA RESULTANTE
modelo.reduce_topics(data, nr_topics=20)
modelo.visualize_topics()

2023-11-14 18:09:16,245 - BERTopic - Reduced number of topics from 22 to 20


De nuevo se obtiene la información general sobre los 20 tópicos resultantes. Se observa que el 1, 17 y 18 corresponden a acumulación de tweets "basura", por lo que no se tendrán en cuenta para el posterior análisis

In [ ]:
modelo.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,192,-1_oniontsla_tslaentry_illegaltsla_caching,"[oniontsla, tslaentry, illegaltsla, caching, t...","[oniontsla, oniontsla, oniontsla]"
1,0,187446,0_trade_stock_nflx_model,"[trade, stock, nflx, model, nvda, short, elon,...",[dont miss next free option trade sign daily f...
2,1,992,1_nan_nano_ching_yeah,"[nan, nano, ching, yeah, , , , , , ]","[nan, nan, nan]"
3,2,772,2_cnbc_cramer_cnbcfastmoney_stock,"[cnbc, cramer, cnbcfastmoney, stock, believer,...","[problem cnbc, cnbc time, cnbc]"
4,3,627,3_bagholderquotes_tent_baggie_gifford,"[bagholderquotes, tent, baggie, gifford, bagho...","[built tent, bagholderquotes, good bagholderqu..."
5,4,281,4_lidar_vaporware_rendering_render,"[lidar, vaporware, rendering, render, product,...",[committed lidar useless dumping lidar cheap s...
6,5,200,5_lotto_call_put_friday,"[lotto, call, put, friday, play, ticket, bough...","[lotto lotto, lotto, lotto]"
7,6,133,6_q32019_q3earnings_qtrresearch_model,"[q32019, q3earnings, qtrresearch, model, produ...",[opportunity increase demand q3earnings q32019...
8,7,110,7_vwap_bwaa_bwaaa_break,"[vwap, bwaa, bwaaa, break, entry, short, long,...","[covered vwap held vwap bounce, vwap call, vwa..."
9,8,110,8_advice_investment_long_providing,"[advice, investment, long, providing, give, sh...","[free advice worth, give advice, great advice]"


Se guarda el modelo con los 20 tópicos resultantes para no perderlo cuando expire la sesión

In [ ]:
modelo.save('/content/drive/MyDrive/TFG ADE/teslatopic20')

Se obtene la información sobre los documentos que sirvieron como base para la categorización de gtópicos, incluyendo los documentos en sí, sus respectivos temas, el nombre de cada tema, las n palabras principales de cada tema, si es un documento representativo y la probabilidad de agrupación si se trata del modelo de agrupación

In [ ]:
modelo.get_document_info(data)

,Document,Topic,Name,Representation,Representative_Docs,Top_n_words,Probability,Representative_document
0,patent portfolio company combined self driving...,0,0_trade_stock_nflx_model,"[trade, stock, nflx, model, nvda, short, elon,...",[dont miss next free option trade sign daily f...,trade - stock - nflx - model - nvda - short - ...,0.000000,False
1,joint development driving system high automati...,0,0_trade_stock_nflx_model,"[trade, stock, nflx, model, nvda, short, elon,...",[dont miss next free option trade sign daily f...,trade - stock - nflx - model - nvda - short - ...,1.000000,False
2,suck never heard dealer able least find equiva...,0,0_trade_stock_nflx_model,"[trade, stock, nflx, model, nvda, short, elon,...",[dont miss next free option trade sign daily f...,trade - stock - nflx - model - nvda - short - ...,0.738882,False
3,elonmusk autopilot almost another twice servic...,0,0_trade_stock_nflx_model,"[trade, stock, nflx, model, nvda, short, elon,...",[dont miss next free option trade sign daily f...,trade - stock - nflx - model - nvda - short - ...,0.000000,False
4,paid spread,0,0_trade_stock_nflx_model,"[trade, stock, nflx, model, nvda, short, elon,...",[dont miss next free option trade sign daily f...,trade - stock - nflx - model - nvda - short - ...,0.000000,False
...,...,...,...,...,...,...,...,...
191593,getting wasted thanks nhtsagov approved autopi...,0,0_trade_stock_nflx_model,"[trade, stock, nflx, model, nvda, short, elon,...",[dont miss next free option trade sign daily f...,trade - stock - nflx - model - nvda - short - ...,0.879340,False
191594,interesting dude,0,0_trade_stock_nflx_model,"[trade, stock, nflx, model, nvda, short, elon,...",[dont miss next free option trade sign daily f...,trade - stock - nflx - model - nvda - short - ...,1.000000,False
191595,2020 brought selfawareness autopilot asshole s...,0,0_trade_stock_nflx_model,"[trade, stock, nflx, model, nvda, short, elon,...",[dont miss next free option trade sign daily f...,trade - stock - nflx - model - nvda - short - ...,0.000000,False
191596,2020 start tweeting positive news mcep every n...,0,0_trade_stock_nflx_model,"[trade, stock, nflx, model, nvda, short, elon,...",[dont miss next free option trade sign daily f...,trade - stock - nflx - model - nvda - short - ...,0.000000,False


Se actualizan las etiquetas estándar de los tópicos resultantes asignadas por el modelo a unas etiquetas custom que faciliten la interpretabilidad de los mismos

In [ ]:
#custom labels
labels_tesla={0: "Inversiones y comparativa entre empresas",
        2: "Análisis CNBC",
        3: "Baillie Gifford",
        4: "Conducción autónoma",
        5: "Lotería",
        6:"Ganancias Q3 2019",
              7:"Comparativa BorgWarner",
              8:"Consejos de inversión",
              9:"Denuncias y acusaciones",
              10:"Range Anxiety",
              11:"Consolidación de acciones",
              12:"Coches eléctricos",
              13:"Especulación CNBC",
              14:"Animadores",
              15:"Lavado de coches",
              16:"Postureo ético"}
modelo.set_topic_labels(labels_tesla)

Se representan en gráficos de barras los 16 tópicos ordenados por relevancia, en los cuales se obtienen los términos más repetidos por tópico y su puntuación c-TF-IDF

In [ ]:
modelo.visualize_barchart(top_n_topics=15,topics=[0,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16], custom_labels=True, width=400)

Se representa la matriz de similitud entre los 10 primeros tópicos más relevantes, para comprobar si son lo suficientemente distintos (valores de similitud menor o igual a 0.5)

In [ ]:
modelo.visualize_heatmap(n_clusters=5, topics=[0,2,3,4,5,6,7,8,9,10],custom_labels=True, width=1200,height=1000)

Se añaden las columnas con la categorización por tópico (número y etiqueta custom) y las palabras clave de cada uno en el dataset

In [ ]:
data_tesla['Tópico']=modelo.get_document_info(data_tesla['body'])['Topic']
data_tesla['Categoría']=modelo.get_document_info(data_tesla['body'])['CustomName']
data_tesla['Palabras clave']=modelo.get_document_info(data_tesla['body'])['Top_n_words']

Se descartan los tweets que tratan sobre el tópico -1 (outlier), y los que acumulan tweets "basura" (1,17,18)

In [ ]:
#se descartan los tweets con el outlier y los tópicos con tweets basura
data_tesla.drop(data_tesla[(data_tesla['Tópico'] ==-1)].index, inplace=True)
data_tesla.drop(data_tesla[(data_tesla['Tópico'] ==1)].index, inplace=True)
data_tesla.drop(data_tesla[(data_tesla['Tópico'] ==17)].index, inplace=True)
data_tesla.drop(data_tesla[(data_tesla['Tópico'] ==18)].index, inplace=True)


Se guarda el dataset con las nuevas columnas

In [ ]:
#se guarda para el analisis de sentimientos
data_tesla.to_csv('/content/drive/MyDrive/TFG ADE/TeslaData20Topics.csv')

In [ ]:
data_tesla.head(50)

,tweet_id,post_date,body,Tópico,Categoría,Palabras clave
0,1134610603715309569,2019-06-01 00:00:22,patent portfolio company combined self driving...,0,Inversiones y comparativa entre empresas,trade - stock - nflx - model - nvda - short - ...
1,1134610685084688385,2019-06-01 00:00:42,joint development driving system high automati...,0,Inversiones y comparativa entre empresas,trade - stock - nflx - model - nvda - short - ...
2,1134610693133611011,2019-06-01 00:00:43,suck never heard dealer able least find equiva...,0,Inversiones y comparativa entre empresas,trade - stock - nflx - model - nvda - short - ...
3,1134610801648689153,2019-06-01 00:01:09,elonmusk autopilot almost another twice servic...,0,Inversiones y comparativa entre empresas,trade - stock - nflx - model - nvda - short - ...
4,1134610879994040320,2019-06-01 00:01:28,paid spread,0,Inversiones y comparativa entre empresas,trade - stock - nflx - model - nvda - short - ...
5,1134610946545127424,2019-06-01 00:01:44,really elon fund recently hopefully spend afte...,0,Inversiones y comparativa entre empresas,trade - stock - nflx - model - nvda - short - ...
6,1134611072411942912,2019-06-01 00:02:14,doesnt sound safe recommendation service cente...,0,Inversiones y comparativa entre empresas,trade - stock - nflx - model - nvda - short - ...
7,1134611235972849664,2019-06-01 00:02:53,instead getting money long though much want ac...,0,Inversiones y comparativa entre empresas,trade - stock - nflx - model - nvda - short - ...
8,1134611314452705280,2019-06-01 00:03:12,great problemunfortunately customer report pro...,0,Inversiones y comparativa entre empresas,trade - stock - nflx - model - nvda - short - ...
9,1134611457981784064,2019-06-01 00:03:46,statement reluctant data,0,Inversiones y comparativa entre empresas,trade - stock - nflx - model - nvda - short - ...


Se obtiene la variación frecuencia por tópico a lo largo de la franja temporal de estudio

In [ ]:
#cambiar a mes
data_tesla['post_date_only']= pd.to_datetime(data_tesla['post_date']).dt.date
modelo = BERTopic.load('/content/drive/MyDrive/TFG ADE/teslatopic20')
topics_over_time = modelo.topics_over_time(data_tesla['body'].astype(str), data_tesla['post_date_only'])

In [ ]:
modelo.visualize_topics_over_time(topics_over_time, topics=[0, 2, 3, 4,5], custom_labels=True, width=1500)

In [ ]:
modelo.visualize_topics_over_time(topics_over_time, topics=[6,7, 8, 9, 10], custom_labels=True, width=1500)

In [ ]:
modelo.visualize_topics_over_time(topics_over_time, topics=[11,12,13, 14, 15, 16], custom_labels=True, width=1500)

Se exporta la evolución temporal por tópico para representarla en otra herramienta

In [ ]:
# evolucion temporal topicos
data_tesla['post_date'] = pd.to_datetime(data_tesla['post_date'])
topics = [0,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]

for topic in topics:
  topic_tweets = data_tesla[data_tesla['Tópico'] == topic]

  # Group the data by day and count the number of tweets per day
  t_per_day = topic_tweets.resample('D', on='post_date').count()
  columns_to_delete = ['body',  'Categoría', 'Palabras clave', 'Unnamed: 0', 'tweet_id']
  t_per_day = t_per_day.drop(columns=columns_to_delete)
  t_per_day = t_per_day.rename(columns={'Tópico': f'Tweets diarios sobre tópico {topic}'})
  t_per_day.to_csv(f'drive/MyDrive/TFG ADE/tesla{topic}.csv')